In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
            
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once at the start
    try:
        for page in range(1, 6):  # There are 5 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
 
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        # Always ensure the driver is quit after scraping
        driver.quit()

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('sb_bus_details.csv', index=False)

print("Scraping complete! Data saved to sb_bus_details.csv.")


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/kolkata-to-durgapur: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6BA7C6CB5+28821]
	(No symbol) [0x00007FF6BA733840]
	(No symbol) [0x00007FF6BA5D578A]
	(No symbol) [0x00007FF6BA6291BE]
	(No symbol) [0x00007FF6BA6294AC]
	(No symbol) [0x00007FF6BA672647]
	(No symbol) [0x00007FF6BA64F33F]
	(No symbol) [0x00007FF6BA66F412]
	(No symbol) [0x00007FF6BA64F0A3]
	(No symbol) [0x00007FF6BA61A778]
	(No symbol) [0x00007FF6BA61B8E1]
	GetHandleVerifier [0x00007FF6BAAFFCAD+3408013]
	GetHandleVerifier [0x00007FF6BAB1741F+3504127]
	GetHandleVerifier [0x00007FF6BAB0B5FD+3455453]
	GetHandleVerifier [0x00007FF6BA88BDBB+835995]
	(No symbol) [0x00007FF6BA73EB5F]
	(No symbol) [0x00007FF6BA73A814]
	(No symbol) [0x00007FF6BA73A9AD]
	(No symbol) [0x00007FF6BA72A199]
	BaseThreadInitThunk [0x00007FFAC59D259D+29]
	RtlUserThreadStart [0x00007FFAC766AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-ti

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),18:00,03h 45m,21:35,4.0,INR 486,7 Seats available
1,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),18:00,03h 45m,21:35,4.1,INR 486,8 Seats available
2,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,Express Line,Volvo 9600 A/C Seater (2+2),18:30,04h 20m,22:50,4.5,INR 486,13 Seats available
3,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KOLKATA - BURDWAN - A/C - 18:45 (BWN DEP...,AC Seater (2+2),18:45,02h 00m,20:45,3.5,INR 180,34 Seats available
4,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KARUNAMOYEE - BURDWAN - A/C - 18:45 (BWN...,AC Seater (2+2),18:45,02h 00m,20:45,3.3,INR 200,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
137,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-SLG-KOL-AC-18:45 - 176648,AC Seater (2+2),19:00,12h 00m,07:00,4.1,INR 800,10 Seats available
138,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,SBSTC-SILIGURI - KOLKATA - VOLVO - 19:35 (BEL ...,Volvo AC Seater (2+2),19:35,13h 00m,08:35,3.5,INR 1210,50 Seats available
139,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Greenline,,18:30,13h 00m,07:30,4.3,INR 1500,22 Seats available
140,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,,,,,,,,
